# "Epidemic modeling - Part 8"
> "Studying real-world data"

- toc: true 
- badges: true
- comments: true
- categories: [modeling, SEIR, epidemiology, stochastic, COVID-19, real-world]
- image: images/goodness-of-fit.png

![](my_icons/goodness-of-fit.png)

In [29]:
#collapse_hide
# This code wrangles the data from JHU
!pip install plotly==4.6.0
!pip install dash==1.12.0
import pandas as pd
import numpy as np

import math

from scipy import signal

import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import expon
from scipy.stats import gamma
from scipy.stats import weibull_min

from numpy.random import default_rng
rng = default_rng()

import dash
import dash_core_components as dcc
import dash_html_components as html

import datetime


# Import confirmed cases
conf_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

#Import deaths data
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Import recovery data
rec_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

#iso_alpha = pd.read_csv('https://raw.githubusercontent.com/jeffufpost/sars-cov-2-world-tracker/master/data/iso_alpha.csv', index_col=0, header=0).T.iloc[0]
iso_alpha = pd.read_csv('https://raw.githubusercontent.com/jeffufpost/sars-cov-2-world-tracker/master/data/iso_alpha.csv', index_col=0, header=0)

# Wrangle the data

#print("Wrangling data by country.......")
# Consolidate countries (ie. frenc dom tom are included in France, etc..)
conf_df = conf_df.groupby("Country/Region")
conf_df = conf_df.sum().reset_index()
conf_df = conf_df.set_index('Country/Region')

deaths_df = deaths_df.groupby("Country/Region")
deaths_df = deaths_df.sum().reset_index()
deaths_df = deaths_df.set_index('Country/Region')

rec_df = rec_df.groupby("Country/Region")
rec_df = rec_df.sum().reset_index()
rec_df = rec_df.set_index('Country/Region')

# Remove Lat and Long columns
conf_df = conf_df.iloc[:,2:]
deaths_df = deaths_df.iloc[:,2:]
rec_df = rec_df.iloc[:,2:]

# Convert country names to correct format for search with pycountry
conf_df = conf_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})
# Convert country names to correct format for search with pycountry
deaths_df = deaths_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})
# Convert country names to correct format for search with pycountry
rec_df = rec_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})

# Convert dates to datime format
conf_df.columns = pd.to_datetime(conf_df.columns).date
deaths_df.columns = pd.to_datetime(deaths_df.columns).date
rec_df.columns = pd.to_datetime(rec_df.columns).date

# Create a per day dataframe
#print("Creating new per day dataframes......")
# Create per day dataframes for cases, deaths, and recoveries - by pd.DatafRame.diff
conf_df_pd = conf_df.diff(axis=1)
deaths_df_pd = deaths_df.diff(axis=1)
rec_df_pd = rec_df.diff(axis=1)

#print("Create infected dataframe = conf - deaths - recoveries")
inf_df = conf_df - deaths_df - rec_df

conf_df_pd.iloc[:,0] = 0
rec_df_pd.iloc[:,0] = 0
deaths_df_pd.iloc[:,0] = 0
inf_df.iloc[:,0] = 0

#print("Adding dataframes of 1st, 2nd, and 3rd derivatives of number of infected")
firstdev = inf_df.apply(np.gradient, axis=1)
seconddev = firstdev.apply(np.gradient)
thirddev = seconddev.apply(np.gradient)

#print("Create series of first date above 100 confirmed cases.....")
# Create a column containing date at which 100 confirmed cases were reached, NaN if not reached yet
fda100 = conf_df[conf_df > 100].apply(pd.Series.first_valid_index, axis=1)

# Create dataframe for probability plot
probevent = iso_alpha.join(inf_df)
probevent['prev'] = probevent.iloc[:,-1] / probevent['SP.POP.TOTL']

In [0]:
#collapse_hide
# This code is the lowpass filter 
def lowpass(x, fc=0.05):
  fs = 1  # Sampling frequency
  t = np.arange(len(x.T)) #select number of days done in SEIR model
  signala = x.T

  #fc = 0.05  # Cut-off frequency of the filter
  w = fc / (fs / 2) # Normalize the frequency
  b, a = signal.butter(5, w, 'low')
  return signal.filtfilt(b, a, signala), w

In [0]:
#collapse_hide
# This code creates the impulse responses
days = np.arange(100)
cdf = pd.DataFrame({
    'T_Latent': gamma.cdf(days, 1.8,loc=0.9,scale=(5.2-1.8)/0.9), 
    'T_Infectious': weibull_min.cdf(days, 2.3,loc=2,scale=20.11)
    })
h_L = cdf.diff().T_Latent
h_I = cdf.diff().T_Infectious
h_L[0] = 0
h_I[0] = 0

In [0]:
#collapse_hide
# This code is for the iterative deconvolution
# Let's define an iteration function:
def iter_deconv(alpha, impulse_response, input_signal, delay, comparator):
  conv=signal.fftconvolve(impulse_response, input_signal, mode='full')
  correction=np.roll(comparator-conv[:len(comparator)], delay)
  input_signal=np.floor(lowpass(alpha*correction+input_signal)[0])
  input_signal[input_signal<0]=0
  return input_signal

# Define a function to return MSE between two signals as a measure of goodness of fit
def msecalc(A, B):
  return ((A - B)**2).mean(axis=0)

## Motivation for write-up

This is the 8th part of a multi-part series blog post on modeling in epidemiology.

The COVID-19 pandemic has brought a lot of attention to the study of epidemiology and more specifically to the various mathematical models that are used to inform public health policies. Everyone has been trying to understand the growth or slowing of new cases and trying to predict the necessary sanitary resources.

While we have studied in detail the stochastc SEIR model, we have not compared this to actual real-world data. 

The goal of this 8th installment is to see how we can use the techniques from the previous blog posts to estimate the various parameters from the real data we have. 

## Data available

We will use the JHU data which is updated daily and displayed graphically on my tracker [here](https://sars-cov-2-world-tracker.herokuapp.com/).

A lot of countries have had difficulty reporting reliable data, but a few have done so rather well.

We will have a closer look at these contries:
* Austria
* Germany
* Iceland
* Russia
* South Korea
* Switzerland

The JHU datasets include the following data:
* Daily cumulative confirmed cases
* Daily cumulative recoveries
* Daily cumulative deaths

From which we can calculate the following:
* Daily new confirmed cases
* Daily new recoveries
* Daily new deaths
* Current number of cases




## Analysis

We have seen previously how we can get $E_{pd}$ from deconvolution of $I_{pd}$.

We also know that:
$$E_{pd} = \beta ~ I ~ \frac{S}{N}$$

Hence:
$$\beta = \frac{N ~ E_{pd}}{I ~ S}$$

And if we assume that in the real-world S \~ N (which is roughly the case where $S=0.9~N$), then:
$$\beta = \frac{E_{pd}}{I}$$

Furthermore we know:
$$R_0 = \frac{\beta}{\gamma}$$ and $$R=R_0\frac{S}{N}$$
So:
$$E_{pd}=\gamma~R~I$$
$$\leftrightarrow R = \frac{E_{pd}}{\gamma~I}$$ 

Our aim is to find this graphically in the real-world data.

## Real-world analysis

### Austria

Let's first have a look at Austria as a template for the analysis we will do for the other countries.

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [202]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Austria'].index, y=conf_df_pd.loc['Austria']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Austria'].index, y=lowpass(conf_df_pd.loc['Austria'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Austria'].index, y=rec_df_pd.loc['Austria']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Austria'].index, y=lowpass(rec_df_pd.loc['Austria'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Austria'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Austria'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Austria: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ leads $h_I[j]\circledast I_{pd}[j]$ by about 5 days.

There is a 20 day lag between peak $I_{pd}$ and $h_I[j]\circledast I_{pd}[j]$ as is expected since $E[T_I] = 20.11$.

But there is only a 15 day lag between peak $I_{pd}$ and actual $R_{pd}$.

There are a few possibilites for why this is the case, including that maybe we haven't assumed the correct distribution for $T_I$.

However there is another reason why this could be. Testing, especially in the early days, took time, and it took time for a patient showing symptoms before he could be tested. This may simply explain the 5 day difference.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [203]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Austria'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Austria'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Austria'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Austria'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Austria'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 994.1480206933169
Iteration #2: MSE= 223.4606324754608
Iteration #3: MSE= 155.95604332037303
Iteration #4: MSE= 138.69774483406667
Iteration #5: MSE= 94.47398861194564
Iteration #6: MSE= 94.27210338177136
Iteration #7: MSE= 72.93041537376978
Iteration #8: MSE= 76.01675231549818 so we use the result of the previous iteration.


In [208]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
#alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Austria'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Austria'])]))
  i=i+1

55.6161301776942
55.411698307523224
55.6161301776942
55.411698307523224
55.6161301776942
55.411698307523224
55.6161301776942
55.411698307523224
55.6161301776942
55.411698307523224


In [210]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Austria'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Austria'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Austria'].index, y=conf_df_pd.loc['Austria']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Austria'].index, y=lowpass(conf_df_pd.loc['Austria'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Austria: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [211]:
#collapse_hide

# Calculate R
gam = 1/15 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Austria'])]*(1/gam)/inf_df.loc['Austria']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Austria'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Austria'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Austria'].index, y=inf_df.loc['Austria']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Austria: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

### France

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [258]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['France'].index, y=conf_df_pd.loc['France']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['France'].index, y=lowpass(conf_df_pd.loc['France']+deaths_df_pd.loc['France'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['France'].index, y=rec_df_pd.loc['France']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['France'].index, y=lowpass(rec_df_pd.loc['France'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['France'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['France']+deaths_df_pd.loc['France'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{France: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

Data is not very good.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [259]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['France'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['France'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['France'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['France'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['France'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 154260.80035828718
Iteration #2: MSE= 54034.481374599236
Iteration #3: MSE= 53691.31465777912
Iteration #4: MSE= 45144.53386180947
Iteration #5: MSE= 39709.43731711392
Iteration #6: MSE= 37860.12268122137
Iteration #7: MSE= 31523.621058356155
Iteration #8: MSE= 32481.060508017406 so we use the result of the previous iteration.


In [261]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['France'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['France'])]))
  i=i+1

25956.38089132117
25965.124359422345
25956.38089132117
25965.124359422345
25956.38089132117
25965.124359422345
25956.38089132117
25965.124359422345
25956.38089132117
25965.124359422345


In [262]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['France'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['France'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['France'].index, y=conf_df_pd.loc['France']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['France'].index, y=lowpass(conf_df_pd.loc['France'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{France: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [263]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['France'])]*(1/gam)/inf_df.loc['France']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['France'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['France'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['France'].index, y=inf_df.loc['France']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{France: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

After March 15th we see a rapi decline in $R$ until $R<1$ since April 12th, however the last 2 weeks of May have seen an increase and it is now close to 1 in early June.

### Germany

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [221]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Germany'].index, y=conf_df_pd.loc['Germany']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Germany'].index, y=lowpass(conf_df_pd.loc['Germany']+deaths_df_pd.loc['Germany'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Germany'].index, y=rec_df_pd.loc['Germany']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Germany'].index, y=lowpass(rec_df_pd.loc['Germany'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Germany'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Germany']+deaths_df_pd.loc['Germany'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Germany: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ lags $h_I[j]\circledast I_{pd}[j]$ by about 10 days.

Although they are pretty close still.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [222]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Germany'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Germany'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Germany'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Germany'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Germany'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 32342.166054726593
Iteration #2: MSE= 9520.818482545094
Iteration #3: MSE= 5480.074863698488
Iteration #4: MSE= 3736.4520218065554
Iteration #5: MSE= 2918.142469998744
Iteration #6: MSE= 2360.7817893148813
Iteration #7: MSE= 1989.5578460711724
Iteration #8: MSE= 1725.1502755073097
Iteration #9: MSE= 1500.7619212142413
Iteration #10: MSE= 1349.7618585161513
Iteration #11: MSE= 1197.9409125553852
Iteration #12: MSE= 1099.740851284486
Iteration #13: MSE= 994.817986262571
Iteration #14: MSE= 924.3422942105037
Iteration #15: MSE= 844.8573516897526
Iteration #16: MSE= 794.3268717988868
Iteration #17: MSE= 739.8998411142202
Iteration #18: MSE= 701.9484322568269
Iteration #19: MSE= 662.5577082814119
Iteration #20: MSE= 631.0944523504336
Iteration #21: MSE= 602.5541705643316
Iteration #22: MSE= 578.5943011308373
Iteration #23: MSE= 557.771540913623
Iteration #24: MSE= 538.213918985831
Iteration #25: MSE= 523.5865826611572
Iteration #26: MSE= 504.6500345303785
Iteration #27: M

In [232]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Germany'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Germany'])]))
  i=i+1

346.90109659914947
348.7463987321059
346.90109659914947
348.7463987321059
346.90109659914947
348.7463987321059
346.90109659914947
348.7463987321059
346.90109659914947
348.7463987321059


In [233]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Germany'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Germany'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Germany'].index, y=conf_df_pd.loc['Germany']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Germany'].index, y=lowpass(conf_df_pd.loc['Germany'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Germany: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [234]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Germany'])]*(1/gam)/inf_df.loc['Germany']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Germany'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Germany'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Germany'].index, y=inf_df.loc['Germany']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Germany: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see $R<1$ since April 3rd, however the peak $E_{pd}$ is slightly ahead of that March 23rd.

### Iceland

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [235]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Iceland'].index, y=conf_df_pd.loc['Iceland']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Iceland'].index, y=lowpass(conf_df_pd.loc['Iceland']+deaths_df_pd.loc['Iceland'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Iceland'].index, y=rec_df_pd.loc['Iceland']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Iceland'].index, y=lowpass(rec_df_pd.loc['Iceland'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Iceland'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Iceland']+deaths_df_pd.loc['Iceland'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Iceland: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ leads $h_I[j]\circledast I_{pd}[j]$ by about 5 days.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [236]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Iceland'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Iceland'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Iceland'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Iceland'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Iceland'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 7.549921432879913
Iteration #2: MSE= 2.1777016171612535
Iteration #3: MSE= 1.4032075531939128
Iteration #4: MSE= 0.7097902831182038
Iteration #5: MSE= 0.6538394434040017
Iteration #6: MSE= 0.35241350030804025
Iteration #7: MSE= 0.4814099480915245 so we use the result of the previous iteration.


In [237]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Iceland'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Iceland'])]))
  i=i+1

0.22104707759253456
0.329023021163164
0.17949206914108762
0.24168412384812277
0.16826563216815357
0.2356731199869026
0.16926773827689567
0.2251716954513438
0.17950404246036664
0.22699309635497517


In [238]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Iceland'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Iceland'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Russia'].index, y=conf_df_pd.loc['Iceland']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Iceland'].index, y=lowpass(conf_df_pd.loc['Iceland'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Iceland: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [239]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Iceland'])]*(1/gam)/inf_df.loc['Iceland']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Iceland'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Iceland'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Iceland'].index, y=inf_df.loc['Iceland']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Iceland: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see $R<1$ since March 30th and dropped significantly within the last 2 weeks of March.


###Russia

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [0]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Russia'].index, y=conf_df_pd.loc['Russia']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Russia'].index, y=lowpass(conf_df_pd.loc['Russia']+deaths_df_pd.loc['Russia'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Russia'].index, y=rec_df_pd.loc['Russia']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Russia'].index, y=lowpass(rec_df_pd.loc['Russia'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Russia'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Russia']+deaths_df_pd.loc['Russia'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Russia: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ lags $h_I[j]\circledast I_{pd}[j]$ by about 4 days.

Again there are a few possibilites for why this is the case, including that maybe we haven't assumed the correct distribution for $T_I$.

However, it can easily be assumed that deaths are vastly underestimated and are the reason for this.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [0]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Russia'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Russia'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Russia'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Russia'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Russia'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 39099.91006982222
Iteration #2: MSE= 16106.103319769038
Iteration #3: MSE= 11017.503739943835
Iteration #4: MSE= 9211.564403068674
Iteration #5: MSE= 8031.942249915613
Iteration #6: MSE= 7057.11955393414
Iteration #7: MSE= 6659.220896111151
Iteration #8: MSE= 6060.561604177258
Iteration #9: MSE= 5976.364288743826
Iteration #10: MSE= 5525.740411388326
Iteration #11: MSE= 5588.427919549833 so we use the result of the previous iteration.


In [0]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Russia'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Russia'])]))
  i=i+1

4386.5897924660985
4471.91717180048
4384.034207273343
4470.893463246848
4382.898268213621
4471.548314093039
4383.598844116114
4468.01306793667
4381.51864607549
4464.993011420806


In [0]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Russia'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Russia'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Russia'].index, y=conf_df_pd.loc['Russia']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Russia'].index, y=lowpass(conf_df_pd.loc['Russia'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Russia: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [0]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Russia'])]*(1/gam)/inf_df.loc['Russia']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Russia'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Russia'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Russia'].index, y=inf_df.loc['Russia']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Russia: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see $R<1$ since May 12th.

### South Korea

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [240]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Korea, Republic of'].index, y=conf_df_pd.loc['Korea, Republic of']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Korea, Republic of'].index, y=lowpass(conf_df_pd.loc['Korea, Republic of']+deaths_df_pd.loc['Korea, Republic of'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Korea, Republic of'].index, y=rec_df_pd.loc['Korea, Republic of']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Korea, Republic of'].index, y=lowpass(rec_df_pd.loc['Korea, Republic of'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Korea, Republic of'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Korea, Republic of']+deaths_df_pd.loc['Korea, Republic of'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Korea, Republic of: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ lags $h_I[j]\circledast I_{pd}[j]$ by about 2 days.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [241]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Korea, Republic of'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Korea, Republic of'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Korea, Republic of'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Korea, Republic of'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Korea, Republic of'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 1002.4468501850523
Iteration #2: MSE= 322.3563402426279
Iteration #3: MSE= 232.31445613577606
Iteration #4: MSE= 225.86390268488017
Iteration #5: MSE= 179.66814661742652
Iteration #6: MSE= 182.39958075753384 so we use the result of the previous iteration.


In [245]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Korea, Republic of'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Korea, Republic of'])]))
  i=i+1

149.3491402956683
149.80157409402977
149.3549551002663
149.80157409402977
149.3549551002661
149.80157409402977
149.3549551002663
149.80157409402977
149.3549551002661
149.80157409402977


In [246]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Korea, Republic of'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Korea, Republic of'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Korea, Republic of'].index, y=conf_df_pd.loc['Korea, Republic of']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Korea, Republic of'].index, y=lowpass(conf_df_pd.loc['Korea, Republic of'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Korea, Republic of: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [247]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Korea, Republic of'])]*(1/gam)/inf_df.loc['Korea, Republic of']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Korea, Republic of'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Korea, Republic of'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Korea, Republic of'].index, y=inf_df.loc['Korea, Republic of']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Korea, Republic of: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see $R<1$ since early March, but has snce climbed back up to 1 and even slightly above that.

### Switzerland

* We have daily data $I_{pd}$ and $R_{pd}$ (where $R_{pd}$ is the sum of deaths and recoveries)
* We have our assumed $T_L$ and $T_I$

Analysis steps:
1. The first thing we want to check is whether $h_I\circledast I_{pd} [j]$ gives us something close to $R_{pd}$ If not, why not?
2. Can we get an estimated $E_{pd}$ by deconvolution of $I_{pd}$ ?
3. What can that tell us about $R$ and $\beta$ ?

#### 1. Checking $h_I$

In [248]:
#collapse_hide
fig = go.Figure(data=[    
    go.Bar(name='Ipd', x=conf_df_pd.loc['Switzerland'].index, y=conf_df_pd.loc['Switzerland']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=conf_df_pd.loc['Switzerland'].index, y=lowpass(conf_df_pd.loc['Switzerland']+deaths_df_pd.loc['Switzerland'])[0]),
    go.Bar(name='Rpd', x=rec_df_pd.loc['Switzerland'].index, y=rec_df_pd.loc['Switzerland']),
    go.Scatter(name='Rpd=lowpass(Rpd)', x=rec_df_pd.loc['Switzerland'].index, y=lowpass(rec_df_pd.loc['Switzerland'])[0]),
    go.Scatter(name='Rpd=conv(Ipd)', x=conf_df_pd.loc['Switzerland'].index, y=signal.fftconvolve(h_I, conf_df_pd.loc['Switzerland']+deaths_df_pd.loc['Switzerland'], mode='full'))
])

fig.update_layout(
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Switzerland: Actual } R_{pd} \text{ vs. } h_I[j]\circledast I_{pd}[j]$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see the actual $R_{pd}$ leads $h_I[j]\circledast I_{pd}[j]$ by about 7 days.

#### 2. Estimating $E_{pd}$ by deconvolution of $I_{pd}$

In [249]:
#collapse_hide

#Settting up for deconvolution of Ipd

#regularization parameter
alpha=2

# Setup up the resultant Ipd we want to compare our guess with
Ipd=np.floor(lowpass(conf_df_pd.loc['Switzerland'])[0])
Ipd[Ipd<0]=0


# Pad with last value
i=0
while i < 100:
  Ipd=np.append(Ipd, Ipd[-1])
  i=i+1

# Find delay caused by h_L
delay=Ipd.argmax()-signal.fftconvolve(Ipd, h_L, mode='full').argmax()

# We want initial guess to simply be the result of the convolution delayed
initial_guess=np.roll(Ipd,delay)
Enext = initial_guess

# AN array to record MSE between result we want and our iterated guess
mse=np.array([])
mse=np.append(mse, 10000000)
mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Switzerland'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Switzerland'])]))

itercount=0
while mse[-1] < mse[-2]:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  mse=np.append(mse, msecalc(Ipd[:len(conf_df_pd.loc['Russia'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Switzerland'])]))
  print("Iteration #" + str(itercount) +": MSE= "+str(mse[itercount]))
print("Iteration #" + str(itercount+1) +": MSE= "+str(mse[-1])+" so we use the result of the previous iteration.")

Iteration #1: MSE= 1470.6459337193169
Iteration #2: MSE= 394.3484792946301
Iteration #3: MSE= 293.667188764407
Iteration #4: MSE= 241.69003136951196
Iteration #5: MSE= 173.82451248640552
Iteration #6: MSE= 163.79909744743827
Iteration #7: MSE= 122.78528193323216
Iteration #8: MSE= 122.56820945779762
Iteration #9: MSE= 96.98263162163703
Iteration #10: MSE= 99.46363623766511 so we use the result of the previous iteration.


In [255]:
#collapse_hide
# We can keep going the iteration until lowest MSE

#change alpha if you like
alpha=2

i=0
while i < 10:
  itercount=itercount+1
  Enext=iter_deconv(alpha, h_L, Enext, delay, Ipd)
  print(msecalc(Ipd[:len(conf_df_pd.loc['Switzerland'])], signal.fftconvolve(h_L, Enext, mode='full')[:len(conf_df_pd.loc['Switzerland'])]))
  i=i+1

58.62220454752213
58.748515359200475
58.62220454752213
58.748515359200475
58.62220454752213
58.748515359200475
58.62220454752213
58.748515359200475
58.62220454752213
58.748515359200475


In [256]:
#collapse_hide
fig = go.Figure(data=[    
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Switzerland'].index, y=Enext),
    go.Scatter(name='Ipd=conv(deconv(Ipd))', x=inf_df.loc['Switzerland'].index, y=signal.fftconvolve(h_L, Enext, mode='full')),
    go.Bar(name='Ipd', x=inf_df.loc['Switzerland'].index, y=conf_df_pd.loc['Switzerland']),
    go.Scatter(name='Ipd=lowpass(Ipd)', x=inf_df.loc['Switzerland'].index, y=lowpass(conf_df_pd.loc['Switzerland'])[0])
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'Count',
    title={
        'text':r'$\text{Switzerland: Actual } I_{pd} \text{ vs. convolution of deconvolution of } I_{pd}$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see that our estimate for $\hat{E}_{pd}$ must be close to the reality of $E_{pd}$ as $I_{pd}$ is almost identical to $\hat{E}_{pd} \circledast h_L$.

Of course, this holds only as long as our estimate of $h_L$ is close to reality.



#### 3. $\beta$ and $R$ from $E_{pd}$ and $I$

As described above:
$$\beta = \frac{N~E_{pd}}{S~I}$$ and if $S$~$N$, then $$\beta = \frac{E_{pd}}{I}$$
It is easier to simply say :
$$R = \frac{E_{pd}}{\gamma~I}$$

In [257]:
#collapse_hide

# Calculate R
gam = 1/20.11 # As we say gamma is 1/20.11
R = Enext[:len(inf_df.loc['Switzerland'])]*(1/gam)/inf_df.loc['Switzerland']

fig = go.Figure(data=[    
    go.Scatter(name='R', x=inf_df.loc['Switzerland'].index, y=R),
    go.Scatter(name='Epd=deconv(Ipd)', x=inf_df.loc['Switzerland'].index, y=Enext),
    go.Scatter(name='Inf', x=inf_df.loc['Switzerland'].index, y=inf_df.loc['Switzerland']),
])

fig.update_layout(
    
    xaxis_title = 'Day',
    yaxis_title = 'R',
    title={
        'text':r'$\text{Switzerland: R }$',
        'x':0.5,
        'xanchor':'center'
    }
)

fig.show()

We can see $R$ declined rapidly after March 12th to $R<1$ on May 30th, but has since grown back to close to 1.